# Exercise 08

## Changes to the Evacuation Model

To enable exercises about sensing and interaction, the evacuation model has been modified:

* Bug fix in `agent.cooperate` (not turning if there is not more than one other agent in the room)
* New model and agent parameter `maxsight`
* Changes in `agent.explore_fieldofvision` to consider `maxsight`
* New switch `DISTANCE_NOISE` to add noise to the calculation of distances to exits (`* self.model.rng.normal(loc=1.0, scale=2.0)`)
* New model parameters `interact_neumann2`, `interact_moore2` and `interact_swnetwork` and agent parameter `interactionmatrix` (combining the three before-mentioned), which indicate the probability of passing alarm message via the particular topology.
* When one of `interact_neumann2`, `interact_moore2` and `interact_swnetwork` is not `None` initially only one agent knows about the alarm.
* With a probabilty of 0.1 each agent who is not alarmed gets alarmed (otherwise the room would never be evacuated if an agent is not informed).
* Adding `net.NetworkGrid`

## Evaluation Code


In [ ]:
from mesa.batchrunner import batch_run
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

import sys
sys.path.insert(0,'../../abmodel')

from fire_evacuation.model import FireEvacuation
from fire_evacuation.agent import Human


unikcolors = [np.array((80,149,200))/255, np.array((74,172,150))/255,
                                                  np.array((234,195,114))/255, np.array((199,16,92))/255]
uniks = LinearSegmentedColormap.from_list( 'unik', unikcolors)

%matplotlib inline

# Task 2 (Sensing in the evacuation model)

## Subtask 2.2

We can change an agents sensing by limiting the field of vision, which is applied when agents search for exits and others to help. Explore the model's sensitivity towards the `maxsight` parameter with batch runs. Produce a figure (consult e.g. exercise 07 task 2.4) showing final values of `TurnCount` and `Step` for increasing values of `maxsight`. Also conduct further simulations with a higher proportion (e.g. 0.5) of agents _not_ believing alarm. Discuss (i.e. contextualise, compare and explain your observations) the results. (<200 words)

In [ ]:
params = dict(
    floor_size=14,
    human_count=100,
    nervousness_mean = 0.3,
    alarm_believers_prop=0.9,
    maxsight = {2,4,6,8,10,12,14,16},
    seed = range(0,20),
)

results = batch_run(
        FireEvacuation,
        parameters=params,
        data_collection_period = 1,
        iterations = 1,
        max_steps = 500,
)

In [ ]:
data1 = pd.DataFrame(results)[['maxsight', "TurnCount", 'Step', 'seed']].round(decimals=1)
db1 = data1.groupby(['seed','maxsight']).agg("max").groupby(['maxsight']).agg("mean")
db1

In [ ]:
# Some hints to produce a comparison figure

# remove columns not needed
d_sw.drop(["interact_moore", "interact_neumann"], axis=1, inplace=True)

# rename column to align it to other frames
d_sw.rename(columns={"interact_swnetwork": "interact"}, inplace=True)

# merge DataFrames on 'column' using union of keys from both frames 
plotdata = pd.merge(df1, df2, how='outer', on='column', suffixes=("_df1", "_df2"))

# define a column as index (for x-axis of plot)
plotdata.set_index('column', inplace=True)

## Subtask 2.3

Sensing is often not 100% accurate - uncertainty is involved. A way to represent uncertainty in agent-based modelling is to introduce noise - a random factor that reduces accuracy of the perceived value. Implement random noise for sensing the distance between the agent and a certain exit. Use the (existing) switch to enable/disable noise and define a parameter to calibrate the level of noise. Consider agent heterogeneity! Test your implementation. Does it change results significantly (t-test)?

In [ ]:
from IPython.utils import io
        
params = dict(
    floor_size=14,
    human_count=70,
    nervousness_mean = 0.3,
    distancenoise = {False,True},
    distancenoiselevel = 0.4,
    seed = range(100,120),
)

results = batch_run(
        FireEvacuation,
        parameters=params,
        data_collection_period = 1,
        iterations = 1,
        max_steps = 500,
    )
data = pd.DataFrame(results)[['distancenoise', 'Step', 'seed']].round(decimals=1)
db = data.groupby(['seed','distancenoise']).agg("max").groupby(['distancenoise']).agg("mean").reset_index()

## Subtask 2.4

Find two more processes of agents' sensing that could be modelled with uncertainty. How would you introduce uncertainty (provide pseudo code)? Discuss whether you should add this option to the model code (<200 words)!

# Task 3 (Interaction in the evacuation model)

## Subtask 3.1

A new option was added to the model: Assume the fire alarm is broken or not existing, and a single randomly choosen agent gets aware of an incident in the room that requires all to escape. We investigate different ways of interaction in terms of passing the information of fire alarm:

* Propagation in the von-Neumann-neighbourhood 
* Propagation in the Moore-Neighbourhood 
* Propagation on a Small-World-Network (Watts-Beta) 

The model parameters determine the probability by which the information is 	passed from an informed agent to its neighbours on the particular topology.
Execute the code blocks in the notebook and compare the results in a figure (!) and interprete them! For small-world networks, why is the <ins>difference</ins> between low values of 	propagation probability higher than for higher values of the probability?

In [ ]:
# Von Neumann:   
params = dict(
    floor_size=14,
    human_count=100,
    nervousness_mean = 0.3,
    cooperation_mean = 0.1,
    interact_neumann = {0.01,0.02,0.05,0.1,0.5,1.0},
    interact_moore = 0.0,
    interact_swnetwork = 0.0,
    seed = range(0,30),
)

results = batch_run(
        FireEvacuation,
        parameters=params,
        data_collection_period = 1,
        iterations = 1,
        max_steps = 500,
    )
data_vn = pd.DataFrame(results)[['interact_neumann', 'interact_moore','interact_swnetwork',
                              'Step', 'TurnCount', 'seed']].round(decimals=2)
db_vn = data_vn.groupby(['seed','interact_neumann','interact_moore','interact_swnetwork',]).agg("max").\
        groupby(['interact_neumann','interact_moore','interact_swnetwork']).agg("mean")
db_vn

In [ ]:
# Moore:
params = dict(
    floor_size=14,
    human_count=100,
    nervousness_mean = 0.3,
    cooperation_mean = 0.1,
    interact_neumann = 0.0,
    interact_moore = {0.01,0.02,0.05,0.1,0.5,1.0},
    interact_swnetwork = 0.0,
    seed = range(0,30),
)

results = batch_run(
        FireEvacuation,
        parameters=params,
        data_collection_period = 1,
        iterations = 1,
        max_steps = 500,
    )
data_mo = pd.DataFrame(results)[['interact_neumann', 'interact_moore','interact_swnetwork',
                              'Step', 'TurnCount', 'seed']].round(decimals=2)
db_mo = data_mo.groupby(['seed','interact_neumann','interact_moore','interact_swnetwork',]).agg("max").\
        groupby(['interact_neumann','interact_moore','interact_swnetwork']).agg("mean")
db_mo

In [ ]:
# Small-world network:
params = dict(
    floor_size=14,
    human_count=100,
    nervousness_mean = 0.3,
    cooperation_mean = 0.3,
    interact_neumann = 0.0,
    interact_moore = 0.0,
    interact_swnetwork = {0.01,0.02,0.05,0.1,0.5,1.0},
    seed = range(0,30),
)

results = batch_run(
        FireEvacuation,
        parameters=params,
        data_collection_period = 1,
        iterations = 1,
        max_steps = 500,
    )

data_sw = pd.DataFrame(results)[['interact_neumann', 'interact_moore','interact_swnetwork', 
                              'Step', 'TurnCount', 'seed']].round(decimals=2)
db_sw = data_sw.groupby(['seed','interact_neumann','interact_moore','interact_swnetwork',]).agg("max").\
        groupby(['interact_neumann','interact_moore','interact_swnetwork']).agg("mean")
db_sw

In [ ]:
# Some hints to produce a figure

# remove columns not needed
d_sw.drop(["interact_moore", "interact_neumann"], axis=1, inplace=True)

# rename column to align it to other frames
d_sw.rename(columns={"interact_swnetwork": "interact"}, inplace=True)

# merge DataFrames on 'column' using union of keys from both frames 
plotdata = pd.merge(df1, df2, how='outer', on='column', suffixes=("_df1", "_df2"))

# define a column as index (for x-axis of plot)
plotdata.set_index('column', inplace=True)

## Subtask 3.2

Which initial position would be ideal for each single interaction topology (vonNeumann, Moore, Network)? Format your answer either as list or table.

## Subtask 3.3

Implement placing the initial knowing agent at a beneficial position for propagation on the smallworld-network. Consider [this answer](https://stackoverflow.com/a/58392749/3957413)!

First describe your idea here in pseudo code, then implement in model.py at line 342ff. Copy and re-run the last part of 3.1 (values for `interact_swnetwork`).